In [1]:
import pandas as pd
import numpy as np
import datetime
from datetime import datetime, timedelta

In [2]:
db = pd.read_csv('prep_prerequisite_data.csv')
data = pd.read_csv('prep_test_case_data.csv')
selected_cols = [
    'CREATED_TIMESTAMP',
    'FORM_ID',
    'ALIEN_ID',
    'JOB_DESCRIPTION',
    'EMPLOYER_NO',
    'MASTER_FORM_TYPE',
    'MASTER_FORM_STATUS',
    'VALID_UNTIL'
    ]
db = db[selected_cols]
data = data[selected_cols + ['LABEL']]

data['CREATED_TIMESTAMP'] = pd.to_datetime(data['CREATED_TIMESTAMP'])
db['CREATED_TIMESTAMP'] = pd.to_datetime(db['CREATED_TIMESTAMP'])

FileNotFoundError: [Errno 2] No such file or directory: 'prep_prerequisite_data.csv'

In [ ]:
data

,CREATED_TIMESTAMP,FORM_ID,ALIEN_ID,JOB_DESCRIPTION,EMPLOYER_NO,MASTER_FORM_TYPE,MASTER_FORM_STATUS,VALID_UNTIL,LABEL
0,2024-04-27,1,1,กรรมกร,12345678,MT_59,Consider request round1,NaN,Abnormal
1,2024-04-27,2,2,กรรมกร,12345679,MT_59,Consider request round1,2027-04-15 11:11:00.000,Normal
2,2024-04-27,2,3,งานขายของหน้าร้าน,12345679,MT_59,Consider request round1,2027-04-15 11:11:00.000,Normal
3,2024-04-27,2,4,กรรมกร,12345679,MT_59,Consider request round1,2027-04-15 11:11:00.000,Normal
4,2024-04-27,2,5,งานทํามือ,12345679,MT_59,Consider request round1,2027-04-15 11:11:00.000,Normal
...,...,...,...,...,...,...,...,...,...
387,2024-07-28,34,385,กรรมกร,25356721,MT_59,Consider request round1,2027-05-02 11:10:59.995,Normal
388,2024-07-28,34,386,กรรมกร,25356721,MT_59,Consider request round1,2027-05-02 11:10:59.995,Normal
389,2024-07-28,34,387,กรรมกร,25356721,MT_59,Consider request round1,2027-05-02 11:10:59.995,Normal
390,2024-07-28,34,388,กรรมกร,25356721,MT_59,Consider request round1,2027-05-02 11:10:59.995,Normal


In [ ]:
def check_inform_mt_13_exit(data: pd.DataFrame, db: pd.DataFrame) -> dict:
    """
    Check Case TC01: Alien Doesn’t Inform Exit
    - Verify if the status register is MT_59.
    - Filter for the latest date to check if the alien has already informed MT_13_Exit in the db.
    input:
        data: Current data 
        db: Historical data 
    """
    results = {}

    # Create a set of unique ALIEN_IDs for efficiency
    alien_ids = set(data["ALIEN_ID"])

    for alien_id in alien_ids:
        #check status register 
        check_MT59 = "MT_59" in data[data["ALIEN_ID"] == alien_id]["MASTER_FORM_TYPE"].values
        
        if check_MT59:
            # Filter and sort only once
            db_filtered = db[db["ALIEN_ID"] == alien_id].sort_values(by="CREATED_TIMESTAMP", ascending=False)
            
            if not db_filtered.empty:
                # Get the latest record
                latest_record = db_filtered.iloc[0]
                # Check for 'MT_13_EXIT' in 'MASTER_FORM_TYPE'
                check_anomaly = 'MT_13_EXIT' in latest_record['MASTER_FORM_TYPE']
                results[alien_id] = check_anomaly

                print(f'ALIEN_ID {alien_id}: {"normal" if check_anomaly else "abnormal"}')
            else:
                print(f'ALIEN_ID {alien_id}: No records found')
                results[alien_id] = None  # Indicates no data found for this ALIEN_ID
                
        else:
            print(f'ALIEN_ID {alien_id}: Master form type not correct')

    return 'abnormal' if False in results.values() else 'normal'
    
db_case = db[db['FORM_ID'] == 2]
data_case = data[data["FORM_ID"] == 2]
check_anomaly = check_inform_mt_13_exit(data_case, db_case)

ALIEN_ID 2: normal
ALIEN_ID 3: normal
ALIEN_ID 4: normal
ALIEN_ID 5: normal
ALIEN_ID 6: normal
ALIEN_ID 7: normal
ALIEN_ID 8: normal
ALIEN_ID 9: normal
ALIEN_ID 10: normal
ALIEN_ID 11: normal
ALIEN_ID 12: normal


In [ ]:
check_anomaly

'normal'

In [3]:
# Test ID 1
db_case = db[db['FORM_ID'] == 1]
check_anomaly = 'abnormal' if 'MT_13_EXIT' not in db_case['MASTER_FORM_STATUS'].values else 'normal'
print(f'Test ID 1: {check_anomaly}')

NameError: name 'db' is not defined

In [4]:
def check_job_limits(data, form_id, master_form_type, config_case):
    """
    Check if the number of jobs in the data exceeds the specified limits in the config_case.

    Parameters:
    - data (pd.DataFrame): The input data containing job information.
    - form_id (int): The form ID to filter the data.
    - master_form_type (str): The master form type to filter the data.
    - config_case (list of dict): A list of dictionaries containing job configurations with 'job' and 'number' keys.

    Returns:
    - str: 'normal' if all job counts are within limits, 'abnormal' otherwise.
    """
    # Filter data based on form_id and master_form_type
    data_case = data[(data['FORM_ID'] == form_id) & (data['MASTER_FORM_TYPE'] == master_form_type)]
    
    # Group by relevant columns and count the number of aliens per job
    data_case = data_case.groupby(['EMPLOYER_NO', 'FORM_ID', 'JOB_DESCRIPTION']).agg(ALIEN_COUNT=('ALIEN_ID', 'count')).reset_index()
    
    # Check each job configuration
    for job_config in config_case:
        job = job_config['job']
        expected_number = job_config['number']
        
        # Calculate the total count of aliens for the specified job
        job_count = data_case[data_case['JOB_DESCRIPTION'] == job]['ALIEN_COUNT'].sum()
        print(f'job: {job}, expected: {expected_number}, job_count: {job_count}')
        
        # If the job count exceeds the expected number, return 'abnormal'
        if job_count > expected_number:
            return 'abnormal'
    
    # If all job counts are within limits, return 'normal'
    return 'normal'

# Example usage
config_case = [
    {"job": "กรรมกร", "number": 6},
    {"job": "งานขายของหน้าร้าน", "number": 1},
    {"job": "งานทํามือ", "number": 2},
]

# Assuming 'data' is a pandas DataFrame containing the relevant data
# data = pd.read_csv('path_to_your_data.csv')  # Example of loading data

# Call the function with the appropriate parameters
result = check_job_limits(data, form_id=2, master_form_type='MT_59', config_case=config_case)
print(f"Test ID 2: {result}")

NameError: name 'data' is not defined

In [6]:
# Test ID 2 - 5

config_case = [
  {
    "job": "กรรมกร",
    "number": 6
  },
  {
    "job": "งานขายของหน้าร้าน",
    "number": 1
  },
  {
    "job": "งานทํามือ",
    "number": 2
  },
] # Each career cannot be exceed 

# If job not in master form config, then it is 'any' job
data_case = data[data['FORM_ID'] == 2]
data_case = data_case[data_case['MASTER_FORM_TYPE'] == 'MT_59']
data_case = data_case.groupby(['EMPLOYER_NO', 'FORM_ID', 'JOB_DESCRIPTION']).agg(ALIEN_COUNT=('ALIEN_ID', 'count')).reset_index()

matched = True
for job_config in config_case:
    job = job_config['job']
    expected_number = job_config['number']
    
    job_count = data_case[data_case['JOB_DESCRIPTION'] == job]['ALIEN_COUNT'].sum()
    print(f'job: {job}, expected: {expected_number}, job_count: {job_count}')
    
    if job_count > expected_number:
        matched = False
        break

# Determine anomaly status
check_anomaly = 'abnormal' if not matched else 'normal'

print(f"Test ID 2: {check_anomaly}")

job: กรรมกร, expected: 6, job_count: 6
job: งานขายของหน้าร้าน, expected: 1, job_count: 3
Test ID 2: abnormal


In [10]:
data_case

,EMPLOYER_NO,FORM_ID,JOB_DESCRIPTION,ALIEN_COUNT
0,12345679,2,กรรมกร,6
1,12345679,2,งานขายของหน้าร้าน,3
2,12345679,2,งานทํามือ,2


In [37]:
def check_timestamp_condition(data, db, form_id, master_form_type_data, master_form_type_db):
    """
    Check if the timestamp condition is met for the given data and database.

    Parameters:
    - data (pd.DataFrame): The input data containing job information.
    - db (pd.DataFrame): The database containing exit information.
    - form_id (int): The form ID to filter both data and db.
    - master_form_type_data (str): The master form type to filter the data.
    - master_form_type_db (str): The master form type to filter the db.

    Returns:
    - str: 'normal' if the timestamp condition is met, 'abnormal' otherwise.
    """
    # Filter data based on form_id and master_form_type
    data_case = data[(data['FORM_ID'] == form_id) & (data['MASTER_FORM_TYPE'] == master_form_type_data)]
    db_case = db[(db['FORM_ID'] == form_id) & (db['MASTER_FORM_TYPE'] == master_form_type_db)]
    
    # Merge data and db on 'ALIEN_ID'
    merged_id = pd.merge(data_case, db_case, on='ALIEN_ID', how='inner')
    
    # Convert timestamps to datetime
    merged_id['CREATED_TIMESTAMP_x'] = pd.to_datetime(merged_id['CREATED_TIMESTAMP_x'])
    merged_id['VALID_UNTIL_y'] = pd.to_datetime(merged_id['VALID_UNTIL_y'])
    
    # Check timestamp condition
    check_anomaly = (merged_id['CREATED_TIMESTAMP_x'] + pd.Timedelta(days=30)) < merged_id['VALID_UNTIL_y']
    
    # Determine anomaly status
    return 'abnormal' if not check_anomaly.iloc[0] else 'normal'

# Call the function with the appropriate parameters
result = check_timestamp_condition(data, db, form_id=6, master_form_type_data='MT_59', master_form_type_db='MT_13_EXIT')
print(f"Test ID 6: {result}")

Test ID 6: abnormal


In [15]:
# Test ID 6
data_case = data[data['FORM_ID'] == 6]
data_case = data_case[data_case['MASTER_FORM_TYPE'] == 'MT_59']
db_case = db[db['FORM_ID'] == 6]
db_case = db_case[db_case['MASTER_FORM_TYPE'] == 'MT_13_EXIT']
merged_id = pd.merge(data_case, db_case, on='ALIEN_ID', how='inner')
merged_id['CREATED_TIMESTAMP_x'] = pd.to_datetime(merged_id['CREATED_TIMESTAMP_x'])
merged_id['VALID_UNTIL_y'] = pd.to_datetime(merged_id['VALID_UNTIL_y'])

# Check timestamp condition
check_anomaly = (merged_id['CREATED_TIMESTAMP_x'] + pd.Timedelta(days=30)) < merged_id['VALID_UNTIL_y']
check_anomaly = 'abnormal' if not check_anomaly.iloc[0] else 'normal'

print(f"Test ID 6: {check_anomaly}")

Test ID 6: abnormal


In [16]:
check_anomaly

'abnormal'

In [39]:
def check_master_form_status(data, db, form_id, master_form_type_data, required_status):
    """
    Check if the required master form status is present in the merged data.

    Parameters:
    - data (pd.DataFrame): The input data containing job information.
    - db (pd.DataFrame): The database containing status information.
    - form_id (int): The form ID to filter both data and db.
    - master_form_type_data (str): The master form type to filter the data.
    - required_status (str): The required master form status to check in the merged data.

    Returns:
    - str: 'normal' if the required status is present, 'abnormal' otherwise.
    """
    # Filter data based on form_id and master_form_type
    data_case = data[(data['FORM_ID'] == form_id) & (data['MASTER_FORM_TYPE'] == master_form_type_data)]
    db_case = db[db['FORM_ID'] == form_id]
    
    # Merge data and db on 'ALIEN_ID'
    merged_id = pd.merge(data_case, db_case, on='ALIEN_ID', how='inner')
    
    # Check if the required status is present in the merged data
    check_anomaly = 'abnormal' if required_status not in merged_id['MASTER_FORM_STATUS_y'].values else 'normal'
    
    return check_anomaly

result = check_master_form_status(data, db, form_id=7, master_form_type_data='MT_59', required_status='MT_13_EXIT')
print(f'Test ID 7: {result}')

Test ID 7: abnormal


In [3]:
# Test ID 7
data_case = data[data['FORM_ID'] == 7]
data_case = data_case[data_case['MASTER_FORM_TYPE'] == 'MT_59']
db_case = db[db['FORM_ID'] == 7]
merged_id = pd.merge(data_case, db_case, on='ALIEN_ID', how='inner', suffixes=('_A', '_B'))
check_anomaly = 'abnormal' if 'MT_13_EXIT' not in merged_id['MASTER_FORM_STATUS_B'].values else 'normal'

print(f'Test ID 7: {check_anomaly}')

Test ID 7: abnormal


In [5]:
db_case

,CREATED_TIMESTAMP,FORM_ID,ALIEN_ID,JOB_DESCRIPTION,EMPLOYER_NO,MASTER_FORM_TYPE,MASTER_FORM_STATUS,VALID_UNTIL
42,2022-04-27,7,42,งานทํารองเท้า,12346803,MT_59,Consider requuest round1,2027-04-15 11:11:00.000
43,2022-04-27,7,43,งานทําหมวก,12346803,MT_59,Consider requuest round1,2027-04-15 11:11:00.000
44,2022-04-27,7,44,กรรมกร,12346803,MT_59,Consider requuest round1,2027-04-15 11:11:00.000


In [4]:
data_case

,CREATED_TIMESTAMP,FORM_ID,ALIEN_ID,JOB_DESCRIPTION,EMPLOYER_NO,MASTER_FORM_TYPE,MASTER_FORM_STATUS,VALID_UNTIL,LABEL
42,2024-06-27,7,42,งานทํารองเท้า,12346802,MT_59,Consider request round1,2027-04-15 11:11:00.000,Abnormal
43,2024-06-27,7,43,งานทําหมวก,12346802,MT_59,Consider request round1,2027-04-15 11:11:00.000,Abnormal
44,2024-06-27,7,44,กรรมกร,12346802,MT_59,Consider request round1,2027-04-15 11:11:00.000,Abnormal
45,2024-06-27,7,44,กรรมกร,12346802,MT_59,Consider request round1,2027-04-15 11:11:00.000,Normal


In [40]:
def check_form_status(db, form_id, required_status):
    """
    Check if the required master form status is present in the database for a given form ID.

    Parameters:
    - db (pd.DataFrame): The database containing status information.
    - form_id (int): The form ID to filter the db.
    - required_status (str): The required master form status to check in the db.

    Returns:
    - str: 'normal' if the required status is not present, 'abnormal' otherwise.
    """
    # Filter db based on form_id
    db_case = db[db['FORM_ID'] == form_id]
    
    # Check if the required status is present in the db
    check_anomaly = 'normal' if required_status not in db_case['MASTER_FORM_STATUS'].values else 'abnormal'
    
    return check_anomaly

result = check_form_status(db, form_id=8, required_status='MT_13_EXIT')
print(f'Test ID 8: {result}')

Test ID 8: normal


In [18]:
# Test ID 8
db_case = db[db['FORM_ID'] == 8]
check_anomaly = 'normal' if 'MT_13_EXIT' not in db_case['MASTER_FORM_STATUS'].values else 'abnormal'
print(f'Test ID 8: {check_anomaly}')

Test ID 8: normal


In [132]:
import pandas as pd
import random
from datetime import datetime, timedelta

config_company = [
    {
        "EMPLOYER_ID": '"010"',
        "COMPANYNAME_TH": '"เจ๋ง ไม่ จำกัด"',
        "COMPANYNAME_EN": '"Jeng Mai Co., Ltd."',
        "BUS_TYPE_ID": '9',
        "EMPLOYER_NO": '"12346803"'
    },
    {
        "EMPLOYER_ID": '"005"',
        "COMPANYNAME_TH": '"ฮา ไม่ จำกัด"',
        "COMPANYNAME_EN": '"Hah Mai Co., Ltd."',
        "BUS_TYPE_ID": '9',
        "EMPLOYER_NO": '"12346800"'
    },
    {
        "EMPLOYER_ID": '"025"',
        "COMPANYNAME_TH": '"ทันโลก ไม่ จำกัด"',
        "COMPANYNAME_EN": '"Tun Lok Co., Ltd."',
        "BUS_TYPE_ID": '17',
        "EMPLOYER_NO": '"25356721"'
    },
    # {
    #     'EMPLOYER_ID': '"020"',
    #     'COMPANYNAME_TH': '"มั่นใจ ไม่ จำกัด"',
    #     'COMPANYNAME_EN': '"Munjai Mai Co., Ltd."',
    #     'BUS_TYPE_ID': '17',
    # },
]

# Function to generate mock data for multiple days for the same company, starting from the end date and moving backward
def generate_mock_data_exit(end_date, num_days):
    data = []
    alien_id_start = 501  # Starting from 501 based on provided example
    count = 0
    for i in range(num_days):
        current_date = end_date - timedelta(days=i)
        date_str = current_date.strftime("%d/%m/%Y %H:%M:%S")
        
        # Generate a random number of employees for each row
        num_employees = random.randint(1, 12)
        
        # Generate lists of ALIEN_ID and ALIEN_SEQ
        alien_ids = ",".join([f'"{alien_id_start + j:05d}"' for j in range(num_employees)])
        alien_seq = ",".join([f'{x}' for x in range(1, num_employees + 1)])
        alien_id_start += num_employees
        
        # Use only "กรรมกร" for job descriptions
        job_descriptions = ",".join(['"กรรมกร"' for _ in range(num_employees)])
        
        # Select a random company configuration
        company = random.choice(config_company)
        
        row = {
            'CREATED_TIMESTAMP': date_str,
            'FORM_ID': f'"{i + 501:05d}"',
            'FORM_ID_SEQ': f'{i + 1}',
            'ALIEN_ID': alien_ids,
            'ALIEN_SEQ': alien_seq,
            'EMPLOYER_ID': company['EMPLOYER_ID'],
            'COMPANYNAME_TH': company['COMPANYNAME_TH'],
            'COMPANYNAME_EN': company['COMPANYNAME_EN'],
            'BUS_TYPE_ID': company['BUS_TYPE_ID'],
            'Job_Description': job_descriptions,
            'EMPLOYER_NO': company['EMPLOYER_NO'],
            'MASTER_FORM_TYPE': 'MT_13_EXIT',
            'MASTER_FORM_STATUS': 'Consider request round1',
            'VALID_UNTIL': '26/06/2024 00:00:00',
        }
        data.append(row)
        count += num_employees
    print(count)
    
    return pd.DataFrame(data)


# Example usage
end_date = datetime.strptime("26/03/2024", "%d/%m/%Y")
num_days = 14
db_exit = generate_mock_data_exit(end_date, num_days)


85


In [133]:
db_exit.to_csv("db_exit_mock.csv", index = False)

In [134]:
# Function to generate mock data for multiple days for the same company, starting from the end date and moving backward
def generate_mock_data(end_date, num_days):
    data = []
    alien_id_start = 501  # Starting from 88 based on provided example
    count = 0
    for i in range(num_days):
        current_date = end_date - timedelta(days=i)
        date_str = current_date.strftime("%d/%m/%Y %H:%M:%S")
        # Generate a random number of employees for each row
        num_employees = random.randint(1, 10)
        
        # Generate lists of ALIEN_ID and ALIEN_SEQ
        alien_ids = ",".join([f'"{alien_id_start + j:05d}"' for j in range(num_employees)])
        alien_seq = ",".join([f'{x}' for x in range(1, num_employees + 1)])
        alien_id_start += num_employees
        
        # Use only "กรรมกร" for job descriptions
        job_descriptions = ",".join(['"กรรมกร"' for _ in range(num_employees)])
        
        row = {
            'CREATED_TIMESTAMP': f'{date_str}',
            'FORM_ID': f'"{i+590:05d}"',
            'FORM_ID_SEQ': f'{i + 1}',
            'ALIEN_ID': alien_ids,
            'ALIEN_SEQ': alien_seq,
            'EMPLOYER_ID': '"023"',
            'COMPANYNAME_TH': '"สุดยอด ไม่ จำกัด"',
            'COMPANYNAME_EN': '"Sud Yod Mai Co., Ltd."',
            'BUS_TYPE_ID': '17',
            'Job_Description': job_descriptions,
            'EMPLOYER_NO': '"23152821"',
            'MASTER_FORM_TYPE': 'MT_59',
            'MASTER_FORM_STATUS': 'Consider request round1',
            'VALID_UNTIL': '26/06/2024 00:00:00',
        }
        data.append(row)
        count += num_employees
    print(count)
    return pd.DataFrame(data)

# Generate mock data
end_date = datetime.strptime("26/06/2024", "%d/%m/%Y")
num_days = 14
db_access = generate_mock_data(end_date, num_days)

77


In [135]:
db_access.to_csv("db_access_mock.csv", index = False)

In [94]:
db_list

<bound method IndexOpsMixin.tolist of 0         1
1       1,2
2     1,2,3
3     1,2,3
4         1
5       1,2
6       1,2
7       1,2
8       1,2
9         1
10    1,2,3
11      1,2
12    1,2,3
13        1
14        1
15    1,2,3
16    1,2,3
17      1,2
18        1
19        1
Name: ALIEN_SEQ, dtype: object>

In [8]:
def prep_data(df):

    # Rename columns
    # df.rename(columns={
    #             'CREATED_TIMESTAMP\n(วันที่ยื่นคำขอ)': 'CREATED_TIMESTAMP',
    #             'FORM_ID\n(รหัสฟอร์ม)': 'FORM_ID',
    #             'FORM_ID_SEQ\n(ลำดับรหัสฟอร์ม)': 'FORM_ID_SEQ',
    #             'ALIEN_ID\n(รหัสคนต่างด้าว)': 'ALIEN_ID',
    #             'ALIEN_SEQ\n(ลำดับรหัสคนต่างด้าว)': 'ALIEN_SEQ',
    #             'EMPLOYER_ID\n(รหัสนายจ้าง)': 'EMPLOYER_ID',
    #             'COMPANYNAME_TH\n(ชื่อนายจ้างไทย/สถานประกอบการ)': 'COMPANYNAME_TH',
    #             'COMPANYNAME_EN\n(ชื่อบริษัท (อังกฤษ))': 'COMPANYNAME_EN',
    #             'BUS_TYPE_ID\n(รหัสประเภทกิจการ)': 'BUS_TYPE_ID',
    #             'Job Description\n(ตำแหน่งงานลูกจ้าง)': 'Job_Description',
    #             'EMPLOYER_NO\n(เลขปชช./นิติบุคคล)': 'EMPLOYER_NO',
    #             'Master_Form_Type\n(FORM_TYPE_ID)': 'Master_Form_Type',
    #             'Master_Form_Status\n(Tracking_Status)': 'Master_Form_Status',
    #                }, inplace=True)
    df.columns = [x.upper() for x in df.columns]

    # Convert columns to datetime
    df['CREATED_TIMESTAMP'] = pd.to_datetime(df['CREATED_TIMESTAMP'])
    df['CREATED_TIMESTAMP'] = df['CREATED_TIMESTAMP'].dt.date

    # Split and explode columns ALIEN_ID, ALIEN_SEQ, Job Description
    df['ALIEN_ID'] = df['ALIEN_ID'].str.split(',')
    df['ALIEN_SEQ'] = df['ALIEN_SEQ'].apply(lambda x: x.split(',') if isinstance(x, str) and ',' in x else [x])
    df['JOB_DESCRIPTION'] = df['JOB_DESCRIPTION'].str.split(',')

    # Explode the columns
    exploded_A = df.explode('ALIEN_ID').reset_index(drop=True)
    exploded_B = df.explode('ALIEN_SEQ').reset_index(drop=True)
    exploded_C = df.explode('JOB_DESCRIPTION').reset_index(drop=True)
    
    # Combine the exploded columns back into a single DataFrame
    exploded_df = exploded_A.copy()
    exploded_df['ALIEN_SEQ'] = exploded_B['ALIEN_SEQ']
    exploded_df['JOB_DESCRIPTION'] = exploded_C['JOB_DESCRIPTION']

    # Fill NaN values with previous values
    exploded_df = exploded_df.ffill()

    def remove_quote(x):
        return x.replace('"', '') if isinstance(x, str) else x

    # List of columns to process
    columns_to_process = [
        'ALIEN_ID', 'FORM_ID', 'EMPLOYER_ID', 
        'COMPANYNAME_TH', 'COMPANYNAME_EN', 
        'BUS_TYPE_ID', 'EMPLOYER_NO', 'JOB_DESCRIPTION'
    ]

    # Apply remove_quote function to each column in the list
    for column in columns_to_process:
        exploded_df[column] = exploded_df[column].apply(remove_quote)

    return exploded_df

In [11]:
# # Test ID 16
config_case = {
    "number": 100,
    "day": 90
    }

# db = pd.read_csv('prep_data_flow4.csv')
data = pd.read_csv('prep_test_case_data.csv')
selected_cols = [
    'CREATED_TIMESTAMP',
    'FORM_ID',
    'ALIEN_ID',
    'JOB_DESCRIPTION',
    'EMPLOYER_NO',
    'MASTER_FORM_TYPE',
    'MASTER_FORM_STATUS',
    'VALID_UNTIL'
    ]
db = db[selected_cols]
data = data[selected_cols + ['LABEL']]

data_case = data[(data['FORM_ID'] == 22) & (data['MASTER_FORM_TYPE'] == 'MT_59')]
# db_case = prep_data(db)
db_case = db

In [12]:
db

,CREATED_TIMESTAMP,FORM_ID,ALIEN_ID,JOB_DESCRIPTION,EMPLOYER_NO,MASTER_FORM_TYPE,MASTER_FORM_STATUS,VALID_UNTIL
0,2022-04-27,1,1,กรรมกร,12346803,MT_59,Consider request round1,2027-04-14 11:10:59.995
1,2024-04-15,2,2,กรรมกร,12346800,MT_13_EXIT,Completed,2027-04-15 11:11:00.000
2,2024-04-15,2,3,งานขายของหน้าร้าน,12346800,MT_13_EXIT,Completed,2027-04-15 11:11:00.000
3,2024-04-15,2,4,กรรมกร,12346800,MT_13_EXIT,Completed,2027-04-15 11:11:00.000
4,2024-04-15,2,5,งานทํามือ,12346800,MT_13_EXIT,Completed,2027-04-15 11:11:00.000
...,...,...,...,...,...,...,...,...
387,2024-05-28,34,385,กรรมกร,20152921,MT_13_EXIT,Completed,2027-05-02 11:10:59.995
388,2024-05-28,34,386,กรรมกร,20152921,MT_13_EXIT,Completed,2027-05-02 11:10:59.995
389,2024-05-28,34,387,กรรมกร,20152921,MT_13_EXIT,Completed,2027-05-02 11:10:59.995
390,2024-05-28,34,388,กรรมกร,20152921,MT_13_EXIT,Completed,2027-05-02 11:10:59.995


In [13]:
data_case = data_case.groupby(['CREATED_TIMESTAMP', 'EMPLOYER_NO', 'FORM_ID']).agg(ALIEN_COUNT=('ALIEN_ID', 'count')).reset_index()

In [14]:
db_case = db_case.groupby(['CREATED_TIMESTAMP', 'EMPLOYER_NO', 'FORM_ID']).agg(ALIEN_COUNT=('ALIEN_ID', 'count')).reset_index()

In [15]:
concat_df = pd.concat([db_case, data_case], axis=0).reset_index(drop=True)

In [16]:
concat_df.sort_values(by = "ALIEN_COUNT", ascending= False)

,CREATED_TIMESTAMP,EMPLOYER_NO,FORM_ID,ALIEN_COUNT
26,2024-05-16,20152921,33,22
33,2024-05-27,23095701,32,20
30,2024-05-20,23095701,31,20
21,2024-05-01,12346993,26,17
29,2024-05-19,12345700,28,17
28,2024-05-18,21152921,25,17
25,2024-05-16,12346800,21,17
23,2024-05-09,12345700,30,16
22,2024-05-04,21152921,20,16
24,2024-05-11,21152921,27,16


In [16]:
# Define the check_scenario function
def check_scenario(df, end_date, limit_count, limit_days):
    # Ensure CREATED_TIMESTAMP is in datetime format
    df['CREATED_TIMESTAMP'] = pd.to_datetime(df['CREATED_TIMESTAMP'])

    # If end_date is a Timestamp, convert it to datetime; otherwise, parse it
    if isinstance(end_date, pd.Timestamp) or isinstance(end_date, pd.DatetimeIndex):
        end_date = end_date.to_pydatetime()
    else:
        end_date = datetime.strptime(end_date, '%Y-%m-%d')
    
    # Calculate the start date for the two-month timeframe
    start_date_2_months = end_date - timedelta(days=60)
    
    # Filter the DataFrame for the two-month timeframe
    df_2_months = df[(df['CREATED_TIMESTAMP'] >= start_date_2_months) & (df['CREATED_TIMESTAMP'] <= end_date)]
    
    # Calculate the total count within the two-month timeframe
    total_count_2_months = df_2_months['ALIEN_COUNT'].sum()
    print(total_count_2_months)

    # Check if the total count within the two-month timeframe exceeds the limit_count
    if total_count_2_months > limit_count:
        # Calculate the start date by subtracting limit_days from end date
        start_date = end_date - timedelta(days=limit_days - 1)
        
        # Filter the DataFrame between the calculated start date and the end date
        filtered_df = df[(df['CREATED_TIMESTAMP'] >= start_date) & (df['CREATED_TIMESTAMP'] <= end_date)]
        
        # Calculate total count and number of days within the limit_days timeframe
        total_count = filtered_df['ALIEN_COUNT'].sum()
        total_days = (filtered_df['CREATED_TIMESTAMP'].max() - filtered_df['CREATED_TIMESTAMP'].min()).days + 1

        # Check scenarios
        if total_count <= limit_count:
            return "ปกติ: Count exceeds limit but days are within limit"
        else:
            if total_days <= limit_days:
                return "ผิดปกติ: Count exceeds limit but days are within limit"
            else:
                return "ปกติ: Count exceeds limit and days exceed limit"
    else:
        return "ปกติ: Total count within two months is within limit"

In [18]:
anomaly = check_scenario(concat_df, concat_df.iloc[-1]['CREATED_TIMESTAMP'], config_case['number'], config_case['day'])
anomaly

267


'ผิดปกติ: Count exceeds limit but days are within limit'

# Test case 09-14

- กรองเฉพาะ mt13-exit, กรองบริษัท ก data_exit
- กรองเฉพาะ mt59, กรองบริษัท ข  data_access
- merge key alien_id 
- gruop by คนที่ย้ายสำเร็จกี่คน by date 
- เข้าฟังก์ชั่น leo  

In [31]:
#create mock up

import pandas as pd
import random
from datetime import datetime, timedelta

config_company = [
    {
        "EMPLOYER_ID": '"010"',
        "COMPANYNAME_TH": '"เจ๋ง ไม่ จำกัด"',
        "COMPANYNAME_EN": '"Jeng Mai Co., Ltd."',
        "BUS_TYPE_ID": '9',
        "EMPLOYER_NO": '"12346803"'
    },
    {
        "EMPLOYER_ID": '"005"',
        "COMPANYNAME_TH": '"ฮา ไม่ จำกัด"',
        "COMPANYNAME_EN": '"Hah Mai Co., Ltd."',
        "BUS_TYPE_ID": '9',
        "EMPLOYER_NO": '"12346800"'
    },
    {
        "EMPLOYER_ID": '"025"',
        "COMPANYNAME_TH": '"ทันโลก ไม่ จำกัด"',
        "COMPANYNAME_EN": '"Tun Lok Co., Ltd."',
        "BUS_TYPE_ID": '17',
        "EMPLOYER_NO": '"25356721"'
    },
    # {
    #     'EMPLOYER_ID': '"020"',
    #     'COMPANYNAME_TH': '"มั่นใจ ไม่ จำกัด"',
    #     'COMPANYNAME_EN': '"Munjai Mai Co., Ltd."',
    #     'BUS_TYPE_ID': '17',
    # },
]

# Function to generate mock data for multiple days for the same company, starting from the end date and moving backward
def generate_mock_data_exit(end_date, num_days):
    data = []
    alien_id_start = 501  # Starting from 501 based on provided example
    
    for i in range(num_days):
        current_date = end_date - timedelta(days=i)
        date_str = current_date.strftime("%d/%m/%Y %H:%M:%S")
        
        # Generate a random number of employees for each row
        num_employees = random.randint(1, 30)
        
        # Generate lists of ALIEN_ID and ALIEN_SEQ
        alien_ids = ",".join([f'"{alien_id_start + j:05d}"' for j in range(num_employees)])
        alien_seq = ",".join([f'{x}' for x in range(1, num_employees + 1)])
        alien_id_start += num_employees
        
        # Use only "กรรมกร" for job descriptions
        job_descriptions = ",".join(['"กรรมกร"' for _ in range(num_employees)])
        
        # Select a random company configuration
        company = random.choice(config_company)
        
        row = {
            'CREATED_TIMESTAMP': date_str,
            'FORM_ID': f'"{i + 501:05d}"',
            'FORM_ID_SEQ': f'{i + 1}',
            'ALIEN_ID': alien_ids,
            'ALIEN_SEQ': alien_seq,
            'EMPLOYER_ID': company['EMPLOYER_ID'],
            'COMPANYNAME_TH': company['COMPANYNAME_TH'],
            'COMPANYNAME_EN': company['COMPANYNAME_EN'],
            'BUS_TYPE_ID': company['BUS_TYPE_ID'],
            'Job_Description': job_descriptions,
            'EMPLOYER_NO': company['EMPLOYER_NO'],
            'MASTER_FORM_TYPE': 'MT_13_EXIT',
            'MASTER_FORM_STATUS': 'Consider request round1',
            'VALID_UNTIL': '26/06/2024 00:00:00',
        }
        data.append(row)
    
    return pd.DataFrame(data)


# Example usage
end_date = datetime.strptime("26/03/2024", "%d/%m/%Y")
num_days = 90
db_exit = generate_mock_data_exit(end_date, num_days)
db_exit = prep_data(db_exit)

# Function to generate mock data for multiple days for the same company, starting from the end date and moving backward
def generate_mock_data(end_date, num_days):
    data = []
    alien_id_start = 501  # Starting from 88 based on provided example
    for i in range(num_days):
        current_date = end_date - timedelta(days=i)
        date_str = current_date.strftime("%d/%m/%Y %H:%M:%S")
        
        # Generate a random number of employees for each row
        num_employees = random.randint(1, 10)
        
        # Generate lists of ALIEN_ID and ALIEN_SEQ
        alien_ids = ",".join([f'"{alien_id_start + j:05d}"' for j in range(num_employees)])
        alien_seq = ",".join([f'{x}' for x in range(1, num_employees + 1)])
        alien_id_start += num_employees
        
        # Use only "กรรมกร" for job descriptions
        job_descriptions = ",".join(['"กรรมกร"' for _ in range(num_employees)])
        
        row = {
            'CREATED_TIMESTAMP': f'{date_str}',
            'FORM_ID': f'"{i+590:05d}"',
            'FORM_ID_SEQ': f'{i + 1}',
            'ALIEN_ID': alien_ids,
            'ALIEN_SEQ': alien_seq,
            'EMPLOYER_ID': '"020"',
            'COMPANYNAME_TH': '"มั่นใจ ไม่ จำกัด"',
            'COMPANYNAME_EN': '"Munjai Mai Co., Ltd."',
            'BUS_TYPE_ID': '17',
            'Job_Description': job_descriptions,
            'EMPLOYER_NO': '"20152921"',
            'MASTER_FORM_TYPE': 'MT_59',
            'MASTER_FORM_STATUS': 'Consider request round1',
            'VALID_UNTIL': '26/06/2024 00:00:00',
        }
        data.append(row)
    
    return pd.DataFrame(data)

# Generate mock data
end_date = datetime.strptime("26/06/2024", "%d/%m/%Y")
num_days = 90
db_access = generate_mock_data(end_date, num_days)
db_access = prep_data(db_access)

/tmp/ipykernel_74539/1160831432.py:22: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['CREATED_TIMESTAMP'] = pd.to_datetime(df['CREATED_TIMESTAMP'])
/tmp/ipykernel_74539/1160831432.py:22: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['CREATED_TIMESTAMP'] = pd.to_datetime(df['CREATED_TIMESTAMP'])


In [33]:
# db_access.to_csv("db_access_flow2.csv", index=False)
# db_exit.to_csv("db_exit_flow2.csv", index=False)

In [52]:
db_exit_prep = pd.read_csv("/opt/project/src/DOE/src/function/db_exit_flow2.csv")
db_access_prep = pd.read_csv("/opt/project/src/DOE/src/function/db_access_flow2.csv")


db_exit_prep["CREATED_TIMESTAMP"] = pd.to_datetime(db_exit_prep['CREATED_TIMESTAMP'])
db_access_prep["CREATED_TIMESTAMP"] = pd.to_datetime(db_access_prep['CREATED_TIMESTAMP'])

In [10]:
# db_exit_prep.to_csv("db_access_flow2.csv", index=False)
# db_access_prep.to_csv("db_exit_flow2.csv", index=False)

In [53]:
config_case = {
    "number": 20,
    "day": 14
    }

In [39]:
merged_alien_id = pd.merge(db_exit_prep, db_access_prep, on='ALIEN_ID', how='inner', suffixes=('_A', '_B'))

In [45]:
db_access_prep.head()

,CREATED_TIMESTAMP,FORM_ID,FORM_ID_SEQ,ALIEN_ID,ALIEN_SEQ,EMPLOYER_ID,COMPANYNAME_TH,COMPANYNAME_EN,BUS_TYPE_ID,JOB_DESCRIPTION,EMPLOYER_NO,MASTER_FORM_TYPE,MASTER_FORM_STATUS,VALID_UNTIL
0,2024-06-26,590,1,501,1,20,มั่นใจ ไม่ จำกัด,"Munjai Mai Co., Ltd.",17,กรรมกร,20152921,MT_59,Consider request round1,26/06/2024 00:00:00
1,2024-06-26,590,1,502,2,20,มั่นใจ ไม่ จำกัด,"Munjai Mai Co., Ltd.",17,กรรมกร,20152921,MT_59,Consider request round1,26/06/2024 00:00:00
2,2024-06-26,590,1,503,3,20,มั่นใจ ไม่ จำกัด,"Munjai Mai Co., Ltd.",17,กรรมกร,20152921,MT_59,Consider request round1,26/06/2024 00:00:00
3,2024-06-26,590,1,504,4,20,มั่นใจ ไม่ จำกัด,"Munjai Mai Co., Ltd.",17,กรรมกร,20152921,MT_59,Consider request round1,26/06/2024 00:00:00
4,2024-06-26,590,1,505,5,20,มั่นใจ ไม่ จำกัด,"Munjai Mai Co., Ltd.",17,กรรมกร,20152921,MT_59,Consider request round1,26/06/2024 00:00:00


In [50]:
db_exit_prep.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1493 entries, 0 to 1492
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   CREATED_TIMESTAMP   1493 non-null   object
 1   FORM_ID             1493 non-null   int64 
 2   FORM_ID_SEQ         1493 non-null   int64 
 3   ALIEN_ID            1493 non-null   int64 
 4   ALIEN_SEQ           1493 non-null   int64 
 5   EMPLOYER_ID         1493 non-null   int64 
 6   COMPANYNAME_TH      1493 non-null   object
 7   COMPANYNAME_EN      1493 non-null   object
 8   BUS_TYPE_ID         1493 non-null   int64 
 9   JOB_DESCRIPTION     1493 non-null   object
 10  EMPLOYER_NO         1493 non-null   int64 
 11  MASTER_FORM_TYPE    1493 non-null   object
 12  MASTER_FORM_STATUS  1493 non-null   object
 13  VALID_UNTIL         1493 non-null   object
dtypes: int64(7), object(7)
memory usage: 163.4+ KB


In [47]:
db_exit_filter = db_exit_prep[(db_exit_prep["EMPLOYER_NO"] == EMPLOYER_NO_A) & (db_exit_prep["MASTER_FORM_TYPE"] == "MT_13_EXIT")]

In [55]:
def TC09(db_access_prep, db_exit_prep, config_case, EMPLOYER_NO_A, EMPLOYER_NO_B):
    #filter mt13-exit เจ๋ง ไม่ จำกัด	
    db_exit_filter = db_exit_prep[(db_exit_prep["EMPLOYER_NO"] == EMPLOYER_NO_A) & (db_exit_prep["MASTER_FORM_TYPE"] == "MT_13_EXIT")]

    #filter mt59 มั่นใจ ไม่ จำกัด
    db_access_filter = db_access_prep[(db_access_prep["EMPLOYER_NO"] == EMPLOYER_NO_B) & (db_access_prep["MASTER_FORM_TYPE"] == "MT_59")]

    #merge key alien_id 
    merged_alien_id = pd.merge(db_exit_filter, db_access_filter, on='ALIEN_ID', how='inner', suffixes=('_A', '_B'))

    print(merged_alien_id)
    #group by raw from date
    count_alien_date = merged_alien_id.groupby(['CREATED_TIMESTAMP_A',"EMPLOYER_NO_A","EMPLOYER_NO_B"]).agg(
        ALIEN_COUNT=('ALIEN_ID', 'count'),  # Count of ALIEN_ID
        ).reset_index().rename(columns={
        'CREATED_TIMESTAMP_A': 'CREATED_TIMESTAMP'})
    
    print(count_alien_date)
    anomaly = check_scenario(count_alien_date, count_alien_date.iloc[-1]['CREATED_TIMESTAMP'], config_case['number'], config_case['day'])
    return anomaly

EMPLOYER_NO_A = 12346803
EMPLOYER_NO_B = 20152921
anomaly, merged_alien_id = TC09(db_access_prep, db_exit_prep, config_case, EMPLOYER_NO_A, EMPLOYER_NO_B)

    CREATED_TIMESTAMP_A  FORM_ID_A  FORM_ID_SEQ_A  ALIEN_ID  ALIEN_SEQ_A  \
0            2024-03-26        501              1       501            1   
1            2024-03-26        501              1       502            2   
2            2024-03-26        501              1       503            3   
3            2024-03-26        501              1       504            4   
4            2024-03-26        501              1       505            5   
..                  ...        ...            ...       ...          ...   
213          2024-03-03        524             24       958           21   
214          2024-03-03        524             24       959           22   
215          2024-03-03        524             24       960           23   
216          2024-03-01        526             26       963            1   
217          2024-03-01        526             26       964            2   

     EMPLOYER_ID_A COMPANYNAME_TH_A    COMPANYNAME_EN_A  BUS_TYPE_ID_A  \
0            

In [22]:
#Test case 09-14
def TC09(data, db, config_case):
    #filter mt13-exit	
    data_exit_filter = data[(data["EMPLOYER_NO"] == "12346803") & (data["MASTER_FORM_TYPE"] == "MT_13_EXIT")]

    #filter mt59 
    db_access_filter = db[(db["EMPLOYER_NO"] == "20152921") & (db["MASTER_FORM_TYPE"] == "MT_59")]

    #merge key alien_id 
    merged_alien_id = pd.merge(data_exit_filter, db_access_filter, on='ALIEN_ID', how='inner', suffixes=('_A', '_B'))

    #group by raw from date
    count_alien_date = merged_alien_id.groupby(['CREATED_TIMESTAMP_A',"EMPLOYER_NO_A","EMPLOYER_NO_B"]).agg(
        ALIEN_COUNT=('ALIEN_ID', 'count'),  # Count of ALIEN_ID
        ).reset_index().rename(columns={
        'CREATED_TIMESTAMP_A': 'CREATED_TIMESTAMP'})
        
    end_date_str = count_alien_date.iloc[-1]['CREATED_TIMESTAMP'].isoformat()
    anomaly = check_scenario(count_alien_date, end_date_str, config_case['number'], config_case['day'])
    return anomaly

TC09(db_access_prep, db_exit_prep, config_case)

IndexError: single positional indexer is out-of-bounds

In [23]:
#filter mt13-exit เจ๋ง ไม่ จำกัด	
db_exit_filter = db_exit_prep[(db_exit_prep["EMPLOYER_NO"] == "12346803") & (db_exit_prep["MASTER_FORM_TYPE"] == "MT_13_EXIT")]

#filter mt59 มั่นใจ ไม่ จำกัด
db_access_filter = db_access_prep[(db_access_prep["EMPLOYER_NO"] == "20152921") & (db_access_prep["MASTER_FORM_TYPE"] == "MT_59")]

#merge key alien_id 
merged_alien_id = pd.merge(db_exit_filter, db_access_filter, on='ALIEN_ID', how='inner', suffixes=('_A', '_B'))

#group by raw from date
count_alien_date = merged_alien_id.groupby(['CREATED_TIMESTAMP_A',"EMPLOYER_NO_A","EMPLOYER_NO_B"]).agg(
    ALIEN_COUNT=('ALIEN_ID', 'count'),  # Count of ALIEN_ID
    ).reset_index().rename(columns={
    'CREATED_TIMESTAMP_A': 'CREATED_TIMESTAMP'})
    
anomaly = check_scenario(count_alien_date, count_alien_date.iloc[-1]['CREATED_TIMESTAMP'].isoformat(), config_case['number'], config_case['day'])

234


In [20]:
anomaly

'ผิดปกติ: Count exceeds limit but days are within limit'

In [12]:
count_alien_date.iloc[-1]['CREATED_TIMESTAMP']

Timestamp('2024-03-19 00:00:00')

In [122]:
type(count_alien_date.iloc[-1]['CREATED_TIMESTAMP'])

pandas._libs.tslibs.timestamps.Timestamp

In [120]:
count_alien_date

,CREATED_TIMESTAMP,EMPLOYER_NO_A,EMPLOYER_NO_B,ALIEN_COUNT
0,2024-02-23,12346803,20152921,23
1,2024-02-28,12346803,20152921,21
2,2024-03-02,12346803,20152921,27
3,2024-03-06,12346803,20152921,2
4,2024-03-08,12346803,20152921,6
5,2024-03-14,12346803,20152921,4
6,2024-03-17,12346803,20152921,8
7,2024-03-24,12346803,20152921,6
8,2024-03-26,12346803,20152921,10


In [114]:
TC09(db_access_prep, db_exit_prep, config_case)

IndexError: single positional indexer is out-of-bounds

In [109]:
count_alien_date

,CREATED_TIMESTAMP,EMPLOYER_NO_A,EMPLOYER_NO_B,ALIEN_COUNT
0,2024-02-23,12346803,20152921,23
1,2024-02-28,12346803,20152921,21
2,2024-03-02,12346803,20152921,27
3,2024-03-06,12346803,20152921,2
4,2024-03-08,12346803,20152921,6
5,2024-03-14,12346803,20152921,4
6,2024-03-17,12346803,20152921,8
7,2024-03-24,12346803,20152921,6
8,2024-03-26,12346803,20152921,10


In [75]:
count_alien_date.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   CREATED_TIMESTAMP  9 non-null      datetime64[ns]
 1   EMPLOYER_NO_A      9 non-null      object        
 2   EMPLOYER_NO_B      9 non-null      object        
 3   ALIEN_COUNT        9 non-null      int64         
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 416.0+ bytes


In [70]:
count_alien_date

,CREATED_TIMESTAMP,EMPLOYER_NO_A,EMPLOYER_NO_B,ALIEN_COUNT
0,2024-02-23,12346803,20152921,23
1,2024-02-28,12346803,20152921,21
2,2024-03-02,12346803,20152921,27
3,2024-03-06,12346803,20152921,2
4,2024-03-08,12346803,20152921,6
5,2024-03-14,12346803,20152921,4
6,2024-03-17,12346803,20152921,8
7,2024-03-24,12346803,20152921,6
8,2024-03-26,12346803,20152921,10


In [110]:
count_alien_date["ALIEN_COUNT"].sum()

107

In [71]:
count_alien_date.columns

Index(['CREATED_TIMESTAMP', 'EMPLOYER_NO_A', 'EMPLOYER_NO_B', 'ALIEN_COUNT'], dtype='object')

In [72]:
count_alien_date.iloc[-1]['CREATED_TIMESTAMP']

datetime.date(2024, 3, 26)

In [104]:
anomaly = check_scenario(count_alien_date, count_alien_date.iloc[-1]['CREATED_TIMESTAMP'], config_case['number'], config_case['day'])
anomaly

107


'ผิดปกติ: Count exceeds limit but days are within limit'

In [74]:
count_alien_date.iloc[-1]['CREATED_TIMESTAMP'].to_pydatetime()

datetime.datetime(2024, 3, 26, 0, 0)

In [38]:
merged_alien_id.columns

Index(['CREATED_TIMESTAMP_A', 'FORM_ID_A', 'FORM_ID_SEQ_A', 'ALIEN_ID',
       'ALIEN_SEQ_A', 'EMPLOYER_ID_A', 'COMPANYNAME_TH_A', 'COMPANYNAME_EN_A',
       'BUS_TYPE_ID_A', 'JOB_DESCRIPTION_A', 'EMPLOYER_NO_A',
       'MASTER_FORM_TYPE_A', 'MASTER_FORM_STATUS_A', 'VALID_UNTIL_A',
       'CREATED_TIMESTAMP_B', 'FORM_ID_B', 'FORM_ID_SEQ_B', 'ALIEN_SEQ_B',
       'EMPLOYER_ID_B', 'COMPANYNAME_TH_B', 'COMPANYNAME_EN_B',
       'BUS_TYPE_ID_B', 'JOB_DESCRIPTION_B', 'EMPLOYER_NO_B',
       'MASTER_FORM_TYPE_B', 'MASTER_FORM_STATUS_B', 'VALID_UNTIL_B'],
      dtype='object')

In [34]:
merged_alien_id

,CREATED_TIMESTAMP_A,FORM_ID_A,FORM_ID_SEQ_A,ALIEN_ID,ALIEN_SEQ_A,EMPLOYER_ID_A,COMPANYNAME_TH_A,COMPANYNAME_EN_A,BUS_TYPE_ID_A,JOB_DESCRIPTION_A,...,ALIEN_SEQ_B,EMPLOYER_ID_B,COMPANYNAME_TH_B,COMPANYNAME_EN_B,BUS_TYPE_ID_B,JOB_DESCRIPTION_B,EMPLOYER_NO_B,MASTER_FORM_TYPE_B,MASTER_FORM_STATUS_B,VALID_UNTIL_B
0,2024-03-26,00501,1,00501,1,010,เจ๋ง ไม่ จำกัด,"Jeng Mai Co., Ltd.",9,กรรมกร,...,1,020,มั่นใจ ไม่ จำกัด,"Munjai Mai Co., Ltd.",17,กรรมกร,20152921,MT_59,Consider request round1,26/06/2024 00:00:00
1,2024-03-26,00501,1,00502,2,010,เจ๋ง ไม่ จำกัด,"Jeng Mai Co., Ltd.",9,กรรมกร,...,2,020,มั่นใจ ไม่ จำกัด,"Munjai Mai Co., Ltd.",17,กรรมกร,20152921,MT_59,Consider request round1,26/06/2024 00:00:00
2,2024-03-26,00501,1,00503,3,010,เจ๋ง ไม่ จำกัด,"Jeng Mai Co., Ltd.",9,กรรมกร,...,3,020,มั่นใจ ไม่ จำกัด,"Munjai Mai Co., Ltd.",17,กรรมกร,20152921,MT_59,Consider request round1,26/06/2024 00:00:00
3,2024-03-26,00501,1,00504,4,010,เจ๋ง ไม่ จำกัด,"Jeng Mai Co., Ltd.",9,กรรมกร,...,4,020,มั่นใจ ไม่ จำกัด,"Munjai Mai Co., Ltd.",17,กรรมกร,20152921,MT_59,Consider request round1,26/06/2024 00:00:00
4,2024-03-26,00501,1,00505,5,010,เจ๋ง ไม่ จำกัด,"Jeng Mai Co., Ltd.",9,กรรมกร,...,5,020,มั่นใจ ไม่ จำกัด,"Munjai Mai Co., Ltd.",17,กรรมกร,20152921,MT_59,Consider request round1,26/06/2024 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102,2024-02-23,00533,33,00994,19,010,เจ๋ง ไม่ จำกัด,"Jeng Mai Co., Ltd.",9,กรรมกร,...,1,020,มั่นใจ ไม่ จำกัด,"Munjai Mai Co., Ltd.",17,กรรมกร,20152921,MT_59,Consider request round1,26/06/2024 00:00:00
103,2024-02-23,00533,33,00995,20,010,เจ๋ง ไม่ จำกัด,"Jeng Mai Co., Ltd.",9,กรรมกร,...,2,020,มั่นใจ ไม่ จำกัด,"Munjai Mai Co., Ltd.",17,กรรมกร,20152921,MT_59,Consider request round1,26/06/2024 00:00:00
104,2024-02-23,00533,33,00996,21,010,เจ๋ง ไม่ จำกัด,"Jeng Mai Co., Ltd.",9,กรรมกร,...,3,020,มั่นใจ ไม่ จำกัด,"Munjai Mai Co., Ltd.",17,กรรมกร,20152921,MT_59,Consider request round1,26/06/2024 00:00:00
105,2024-02-23,00533,33,00997,22,010,เจ๋ง ไม่ จำกัด,"Jeng Mai Co., Ltd.",9,กรรมกร,...,4,020,มั่นใจ ไม่ จำกัด,"Munjai Mai Co., Ltd.",17,กรรมกร,20152921,MT_59,Consider request round1,26/06/2024 00:00:00


In [26]:
db_access_filter

,CREATED_TIMESTAMP,FORM_ID,FORM_ID_SEQ,ALIEN_ID,ALIEN_SEQ,EMPLOYER_ID,COMPANYNAME_TH,COMPANYNAME_EN,BUS_TYPE_ID,JOB_DESCRIPTION,EMPLOYER_NO,MASTER_FORM_TYPE,MASTER_FORM_STATUS,VALID_UNTIL
0,2024-06-26,00590,1,00501,1,020,มั่นใจ ไม่ จำกัด,"Munjai Mai Co., Ltd.",17,กรรมกร,20152921,MT_59,Consider request round1,26/06/2024 00:00:00
1,2024-06-26,00590,1,00502,2,020,มั่นใจ ไม่ จำกัด,"Munjai Mai Co., Ltd.",17,กรรมกร,20152921,MT_59,Consider request round1,26/06/2024 00:00:00
2,2024-06-26,00590,1,00503,3,020,มั่นใจ ไม่ จำกัด,"Munjai Mai Co., Ltd.",17,กรรมกร,20152921,MT_59,Consider request round1,26/06/2024 00:00:00
3,2024-06-26,00590,1,00504,4,020,มั่นใจ ไม่ จำกัด,"Munjai Mai Co., Ltd.",17,กรรมกร,20152921,MT_59,Consider request round1,26/06/2024 00:00:00
4,2024-06-26,00590,1,00505,5,020,มั่นใจ ไม่ จำกัด,"Munjai Mai Co., Ltd.",17,กรรมกร,20152921,MT_59,Consider request round1,26/06/2024 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,2024-03-30,00678,89,01001,2,020,มั่นใจ ไม่ จำกัด,"Munjai Mai Co., Ltd.",17,กรรมกร,20152921,MT_59,Consider request round1,26/06/2024 00:00:00
501,2024-03-30,00678,89,01002,3,020,มั่นใจ ไม่ จำกัด,"Munjai Mai Co., Ltd.",17,กรรมกร,20152921,MT_59,Consider request round1,26/06/2024 00:00:00
502,2024-03-29,00679,90,01003,1,020,มั่นใจ ไม่ จำกัด,"Munjai Mai Co., Ltd.",17,กรรมกร,20152921,MT_59,Consider request round1,26/06/2024 00:00:00
503,2024-03-29,00679,90,01004,2,020,มั่นใจ ไม่ จำกัด,"Munjai Mai Co., Ltd.",17,กรรมกร,20152921,MT_59,Consider request round1,26/06/2024 00:00:00


In [27]:
db_exit_filter

,CREATED_TIMESTAMP,FORM_ID,FORM_ID_SEQ,ALIEN_ID,ALIEN_SEQ,EMPLOYER_ID,COMPANYNAME_TH,COMPANYNAME_EN,BUS_TYPE_ID,JOB_DESCRIPTION,EMPLOYER_NO,MASTER_FORM_TYPE,MASTER_FORM_STATUS,VALID_UNTIL
0,2024-03-26,00501,1,00501,1,010,เจ๋ง ไม่ จำกัด,"Jeng Mai Co., Ltd.",9,กรรมกร,12346803,MT_13_EXIT,Consider request round1,26/06/2024 00:00:00
1,2024-03-26,00501,1,00502,2,010,เจ๋ง ไม่ จำกัด,"Jeng Mai Co., Ltd.",9,กรรมกร,12346803,MT_13_EXIT,Consider request round1,26/06/2024 00:00:00
2,2024-03-26,00501,1,00503,3,010,เจ๋ง ไม่ จำกัด,"Jeng Mai Co., Ltd.",9,กรรมกร,12346803,MT_13_EXIT,Consider request round1,26/06/2024 00:00:00
3,2024-03-26,00501,1,00504,4,010,เจ๋ง ไม่ จำกัด,"Jeng Mai Co., Ltd.",9,กรรมกร,12346803,MT_13_EXIT,Consider request round1,26/06/2024 00:00:00
4,2024-03-26,00501,1,00505,5,010,เจ๋ง ไม่ จำกัด,"Jeng Mai Co., Ltd.",9,กรรมกร,12346803,MT_13_EXIT,Consider request round1,26/06/2024 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1281,2023-12-31,00587,87,01782,25,010,เจ๋ง ไม่ จำกัด,"Jeng Mai Co., Ltd.",9,กรรมกร,12346803,MT_13_EXIT,Consider request round1,26/06/2024 00:00:00
1282,2023-12-31,00587,87,01783,26,010,เจ๋ง ไม่ จำกัด,"Jeng Mai Co., Ltd.",9,กรรมกร,12346803,MT_13_EXIT,Consider request round1,26/06/2024 00:00:00
1283,2023-12-31,00587,87,01784,27,010,เจ๋ง ไม่ จำกัด,"Jeng Mai Co., Ltd.",9,กรรมกร,12346803,MT_13_EXIT,Consider request round1,26/06/2024 00:00:00
1284,2023-12-31,00587,87,01785,28,010,เจ๋ง ไม่ จำกัด,"Jeng Mai Co., Ltd.",9,กรรมกร,12346803,MT_13_EXIT,Consider request round1,26/06/2024 00:00:00


In [140]:
db_mockup = pd.read_excel("/opt/project/src/DOE/src/function/Testcase_DOE_2-7-2024.xlsx", sheet_name= "DB_mockup")
db_mockup.drop(columns= ["Job Anomaly\n(ตำแหน่งงานที่จ้างมากกว่าปกติ)"], inplace = True)

In [141]:
db_mockup.head()

,Test ID,Rule number,Case Description,CREATED_TIMESTAMP\n(วันที่ยื่นคำขอ),FORM_ID\n(รหัสฟอร์ม),FORM_ID_SEQ\n(ลำดับรหัสฟอร์ม),ALIEN_ID\n(รหัสคนต่างด้าว),ALIEN_SEQ\n(ลำดับรหัสคนต่างด้าว),EMPLOYER_ID\n(รหัสนายจ้าง),COMPANYNAME_TH\n(ชื่อนายจ้างไทย/สถานประกอบการ),COMPANYNAME_EN\n(ชื่อบริษัท (อังกฤษ)),BUS_TYPE_ID\n(รหัสประเภทกิจการ),Job Description\n(ตำแหน่งงานลูกจ้าง),EMPLOYER_NO\n(เลขปชช./นิติบุคคล),Master_Form_Type\n(FORM_TYPE_ID),Master_Form_Status\n(Tracking_Status),Valid_Until,data_type
0,Test case 16,4,กรณีปกติ: กลุ่มคนต่างด้าวย้ายมาที่ ข ไม่เกินจำ...,26/06/2024 00:00:00,"""00590""",1,"""00501""",1,"""020""","""มั่นใจ ไม่ จำกัด""","""Munjai Mai Co., Ltd.""",17,"""กรรมกร""","""20152921""",MT_59,Consider request round1,26/06/2024 00:00:00,Prerequisite
1,Test case 16,4,กรณีปกติ: กลุ่มคนต่างด้าวย้ายมาที่ ข ไม่เกินจำ...,25/06/2024 00:00:00,"""00591""",2,"""00502"",""00503"",""00504""",123,"""020""","""มั่นใจ ไม่ จำกัด""","""Munjai Mai Co., Ltd.""",17,"""กรรมกร"",""กรรมกร"",""กรรมกร""","""20152921""",MT_59,Consider request round1,26/06/2024 00:00:00,Prerequisite
2,Test case 16,4,กรณีปกติ: กลุ่มคนต่างด้าวย้ายมาที่ ข ไม่เกินจำ...,24/06/2024 00:00:00,"""00592""",3,"""00505""",1,"""020""","""มั่นใจ ไม่ จำกัด""","""Munjai Mai Co., Ltd.""",17,"""กรรมกร""","""20152921""",MT_59,Consider request round1,26/06/2024 00:00:00,Prerequisite
3,Test case 16,4,กรณีปกติ: กลุ่มคนต่างด้าวย้ายมาที่ ข ไม่เกินจำ...,23/06/2024 00:00:00,"""00593""",4,"""00506""",1,"""020""","""มั่นใจ ไม่ จำกัด""","""Munjai Mai Co., Ltd.""",17,"""กรรมกร""","""20152921""",MT_59,Consider request round1,26/06/2024 00:00:00,Prerequisite
4,Test case 16,4,กรณีปกติ: กลุ่มคนต่างด้าวย้ายมาที่ ข ไม่เกินจำ...,22/06/2024 00:00:00,"""00594""",5,"""00507"",""00508"",""00509""",123,"""020""","""มั่นใจ ไม่ จำกัด""","""Munjai Mai Co., Ltd.""",17,"""กรรมกร"",""กรรมกร"",""กรรมกร""","""20152921""",MT_59,Consider request round1,26/06/2024 00:00:00,Prerequisite


In [142]:
db_mockup.columns

Index(['Test ID', 'Rule number', 'Case Description',
       'CREATED_TIMESTAMP\n(วันที่ยื่นคำขอ)', 'FORM_ID\n(รหัสฟอร์ม)',
       'FORM_ID_SEQ\n(ลำดับรหัสฟอร์ม)', 'ALIEN_ID\n(รหัสคนต่างด้าว)',
       'ALIEN_SEQ\n(ลำดับรหัสคนต่างด้าว)', 'EMPLOYER_ID\n(รหัสนายจ้าง)',
       'COMPANYNAME_TH\n(ชื่อนายจ้างไทย/สถานประกอบการ)',
       'COMPANYNAME_EN\n(ชื่อบริษัท (อังกฤษ))',
       'BUS_TYPE_ID\n(รหัสประเภทกิจการ)',
       'Job Description\n(ตำแหน่งงานลูกจ้าง)',
       'EMPLOYER_NO\n(เลขปชช./นิติบุคคล)', 'Master_Form_Type\n(FORM_TYPE_ID)',
       'Master_Form_Status\n(Tracking_Status)', 'Valid_Until', 'data_type'],
      dtype='object')

In [146]:
def prep_data(df):

    #Rename columns
    df.rename(columns={
                'CREATED_TIMESTAMP\n(วันที่ยื่นคำขอ)': 'CREATED_TIMESTAMP',
                'FORM_ID\n(รหัสฟอร์ม)': 'FORM_ID',
                'FORM_ID_SEQ\n(ลำดับรหัสฟอร์ม)': 'FORM_ID_SEQ',
                'ALIEN_ID\n(รหัสคนต่างด้าว)': 'ALIEN_ID',
                'ALIEN_SEQ\n(ลำดับรหัสคนต่างด้าว)': 'ALIEN_SEQ',
                'EMPLOYER_ID\n(รหัสนายจ้าง)': 'EMPLOYER_ID',
                'COMPANYNAME_TH\n(ชื่อนายจ้างไทย/สถานประกอบการ)': 'COMPANYNAME_TH',
                'COMPANYNAME_EN\n(ชื่อบริษัท (อังกฤษ))': 'COMPANYNAME_EN',
                'BUS_TYPE_ID\n(รหัสประเภทกิจการ)': 'BUS_TYPE_ID',
                'Job Description\n(ตำแหน่งงานลูกจ้าง)': 'Job_Description',
                'EMPLOYER_NO\n(เลขปชช./นิติบุคคล)': 'EMPLOYER_NO',
                'Master_Form_Type\n(FORM_TYPE_ID)': 'Master_Form_Type',
                'Master_Form_Status\n(Tracking_Status)': 'Master_Form_Status',
                   }, inplace=True)
    df.columns = [x.upper() for x in df.columns]

    # Convert columns to datetime
    df['CREATED_TIMESTAMP'] = pd.to_datetime(df['CREATED_TIMESTAMP'])
    df['CREATED_TIMESTAMP'] = df['CREATED_TIMESTAMP'].dt.date

    # Split and explode columns ALIEN_ID, ALIEN_SEQ, Job Description
    df['ALIEN_ID'] = df['ALIEN_ID'].str.split(',')
    df['ALIEN_SEQ'] = df['ALIEN_SEQ'].apply(lambda x: x.split(',') if isinstance(x, str) and ',' in x else [x])
    df['JOB_DESCRIPTION'] = df['JOB_DESCRIPTION'].str.split(',')

    # Explode the columns
    exploded_A = df.explode('ALIEN_ID').reset_index(drop=True)
    exploded_B = df.explode('ALIEN_SEQ').reset_index(drop=True)
    exploded_C = df.explode('JOB_DESCRIPTION').reset_index(drop=True)
    
    # Combine the exploded columns back into a single DataFrame
    exploded_df = exploded_A.copy()
    exploded_df['ALIEN_SEQ'] = exploded_B['ALIEN_SEQ']
    exploded_df['JOB_DESCRIPTION'] = exploded_C['JOB_DESCRIPTION']

    # Fill NaN values with previous values
    exploded_df = exploded_df.ffill()

    def remove_quote(x):
        return x.replace('"', '') if isinstance(x, str) else x

    # List of columns to process
    columns_to_process = [
        'ALIEN_ID', 'FORM_ID', 'EMPLOYER_ID', 
        'COMPANYNAME_TH', 'COMPANYNAME_EN', 
        'BUS_TYPE_ID', 'EMPLOYER_NO', 'JOB_DESCRIPTION'
    ]

    # Apply remove_quote function to each column in the list
    for column in columns_to_process:
        exploded_df[column] = exploded_df[column].apply(remove_quote)

    return exploded_df

In [147]:
db_mockup_prep = prep_data(db_mockup)

/tmp/ipykernel_74539/4281262514.py:22: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['CREATED_TIMESTAMP'] = pd.to_datetime(df['CREATED_TIMESTAMP'])


In [148]:
db_mockup_prep

,TEST ID,RULE NUMBER,CASE DESCRIPTION,CREATED_TIMESTAMP,FORM_ID,FORM_ID_SEQ,ALIEN_ID,ALIEN_SEQ,EMPLOYER_ID,COMPANYNAME_TH,COMPANYNAME_EN,BUS_TYPE_ID,JOB_DESCRIPTION,EMPLOYER_NO,MASTER_FORM_TYPE,MASTER_FORM_STATUS,VALID_UNTIL,DATA_TYPE
0,Test case 16,4,กรณีปกติ: กลุ่มคนต่างด้าวย้ายมาที่ ข ไม่เกินจำ...,2024-06-26,00590,1,00501,1,020,มั่นใจ ไม่ จำกัด,"Munjai Mai Co., Ltd.",17,กรรมกร,20152921,MT_59,Consider request round1,26/06/2024 00:00:00,Prerequisite
1,Test case 16,4,กรณีปกติ: กลุ่มคนต่างด้าวย้ายมาที่ ข ไม่เกินจำ...,2024-06-25,00591,2,00502,123,020,มั่นใจ ไม่ จำกัด,"Munjai Mai Co., Ltd.",17,กรรมกร,20152921,MT_59,Consider request round1,26/06/2024 00:00:00,Prerequisite
2,Test case 16,4,กรณีปกติ: กลุ่มคนต่างด้าวย้ายมาที่ ข ไม่เกินจำ...,2024-06-25,00591,2,00503,1,020,มั่นใจ ไม่ จำกัด,"Munjai Mai Co., Ltd.",17,กรรมกร,20152921,MT_59,Consider request round1,26/06/2024 00:00:00,Prerequisite
3,Test case 16,4,กรณีปกติ: กลุ่มคนต่างด้าวย้ายมาที่ ข ไม่เกินจำ...,2024-06-25,00591,2,00504,1,020,มั่นใจ ไม่ จำกัด,"Munjai Mai Co., Ltd.",17,กรรมกร,20152921,MT_59,Consider request round1,26/06/2024 00:00:00,Prerequisite
4,Test case 16,4,กรณีปกติ: กลุ่มคนต่างด้าวย้ายมาที่ ข ไม่เกินจำ...,2024-06-24,00592,3,00505,123,020,มั่นใจ ไม่ จำกัด,"Munjai Mai Co., Ltd.",17,กรรมกร,20152921,MT_59,Consider request round1,26/06/2024 00:00:00,Prerequisite
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
792,Test case 14,2,กรณีผิดปกติ: กลุ่มคนต่างด้าวย้ายจาก ก ไป ข เกิ...,2024-03-13,00514,14,00581,123456789,010,เจ๋ง ไม่ จำกัด,"Jeng Mai Co., Ltd.",9,กรรมกร,12346803,MT_13_EXIT,Completed,26/06/2025 00:00:00,Prerequisite
793,Test case 14,2,กรณีผิดปกติ: กลุ่มคนต่างด้าวย้ายจาก ก ไป ข เกิ...,2024-03-13,00514,14,00582,123456789,010,เจ๋ง ไม่ จำกัด,"Jeng Mai Co., Ltd.",9,กรรมกร,12346803,MT_13_EXIT,Completed,26/06/2025 00:00:00,Prerequisite
794,Test case 14,2,กรณีผิดปกติ: กลุ่มคนต่างด้าวย้ายจาก ก ไป ข เกิ...,2024-03-13,00514,14,00583,123456789,010,เจ๋ง ไม่ จำกัด,"Jeng Mai Co., Ltd.",9,กรรมกร,12346803,MT_13_EXIT,Completed,26/06/2025 00:00:00,Prerequisite
795,Test case 14,2,กรณีผิดปกติ: กลุ่มคนต่างด้าวย้ายจาก ก ไป ข เกิ...,2024-03-13,00514,14,00584,123456789,010,เจ๋ง ไม่ จำกัด,"Jeng Mai Co., Ltd.",9,กรรมกร,12346803,MT_13_EXIT,Completed,26/06/2025 00:00:00,Prerequisite


In [12]:
# import cx_Oracle
import pandas as pd
from datetime import datetime
import os
from dotenv import load_dotenv
import psycopg2
from sqlalchemy import create_engine


class DatabaseManager:
    def __init__(self):
        load_dotenv()
        self.oracle_username = os.getenv("ORACLE_USERNAME")
        self.oracle_password = os.getenv("ORACLE_PASSWORD")
        self.oracle_host = os.getenv("ORACLE_HOST")
        self.oracle_port = os.getenv("ORACLE_PORT")
        self.oracle_service_name = os.getenv("ORACLE_SERVICE_NAME")
        self.postgres_db = os.getenv("POSTGRES_DB")
        self.postgres_user = os.getenv("POSTGRES_USER")
        self.postgres_password = os.getenv("POSTGRES_PASSWORD")
        self.postgres_host = os.getenv("POSTGRES_SERVER")

    def execute_oracle_query(self, query):
        try:
            dsn = cx_Oracle.makedsn(self.oracle_host, self.oracle_port, service_name=self.oracle_service_name)
            connection = cx_Oracle.connect(user=self.oracle_username, password=self.oracle_password, dsn=dsn)
            print("Successfully connected to Oracle Database")

            cursor = connection.cursor()
            cursor.execute(query)
            records = cursor.fetchall()
            column_names = [col[0] for col in cursor.description]
            df_records = pd.DataFrame(records, columns=column_names)
            # df_records['BATCH'] = datetime.now().strftime('%Y-%m-%d')
            df_records['BATCH'] = '2024-7-10'
            # df_records['BATCH'] = '2024-7-11'
            cursor.close()
            connection.close()

            pd.set_option('display.max_rows', None)
            pd.set_option('display.max_columns', None)
            pd.set_option('display.width', None)
            pd.set_option('display.max_colwidth', None)

            return df_records

        except cx_Oracle.DatabaseError as e:
            print(f"An error occurred: {e}")
            return None

    def create_postgres_table(self):
        try:
            conn = psycopg2.connect(
                dbname=self.postgres_db,
                user=self.postgres_user,
                password=self.postgres_password,
                host=self.postgres_host
            )

            cur = conn.cursor()
            create_table_query = """
            CREATE TABLE IF NOT EXISTS landing.forms (
                FORM_ID VARCHAR,
                FORM_ID_SEQ VARCHAR,
                EMPLOYER_ID VARCHAR,
                JOB_01 VARCHAR,
                FORM_STATUS_ID VARCHAR,
                ALIEN_HAS_PERMIT VARCHAR,
                ALIEN_PERMIT_NO VARCHAR,
                ALIEN_STAY_PERMIS_SEQ VARCHAR,
                END_DT_SLA_DOE TIMESTAMP,
                IS_ACCEPT_WORK VARCHAR,
                BATCH DATE
            );
            """

            cur.execute(create_table_query)
            conn.commit()
            print("Table 'forms' created in the 'landing' schema.")
            cur.close()
            conn.close()

        except psycopg2.DatabaseError as e:
            print(f"An error occurred: {e}")

    def ingest_data(self, query):
        try:
            connection_string = f"postgresql://{self.postgres_user}:{self.postgres_password}@{self.postgres_host}/{self.postgres_db}"
            engine = create_engine(connection_string)
            df = self.execute_oracle_query(query)

            if df is not None and not df.empty:
                df.columns = [col.lower() for col in df.columns]

                # Checking if data already exists
                existing_data = pd.read_sql_table('forms', engine, schema='landing')
                new_data = df[~df.index.isin(existing_data.index)]

                if not new_data.empty:
                    new_data.to_sql('forms', engine, schema='landing', if_exists='append', index=False)
                    print("New data ingested into the 'landing.forms' table.")
                else:
                    print("No new data to ingest.")
            else:
                print("Query returned no data.")

        except Exception as e:
            print(f"An error occurred during ingestion: {e}")

    def query_postgres(self, query):
        try:
            connection_string = f"postgresql://{self.postgres_user}:{self.postgres_password}@{self.postgres_host}/{self.postgres_db}"
            engine = create_engine(connection_string)
            df = pd.read_sql(query, engine)

            pd.set_option('display.max_rows', None)
            pd.set_option('display.max_columns', None)
            pd.set_option('display.width', None)
            pd.set_option('display.max_colwidth', None)
            return df

        except Exception as e:
            print(f"An error occurred: {e}")
            return None
if __name__ == "__main__":
    db_manager = DatabaseManager()
    query = """
        SELECT 
            FORM_ID,
            FORM_ID_SEQ,
            EMPLOYER_ID,
            JOB_01,
            FORM_STATUS_ID,
            ALIEN_HAS_PERMIT,
            ALIEN_PERMIT_NO,
            ALIEN_STAY_PERMIS_SEQ,
            END_DT_SLA_DOE,
            IS_ACCEPT_WORK
        FROM 
            PRD_MOL.FORMS FETCH FIRST 20 ROWS ONLY
    """
    # db_manager.create_postgres_table()
    # db_manager.ingest_data(query)

In [13]:
query = """ 
SELECT *
FROM landing.forms LIMIT 1000
"""
# df = db_manager.query_postgres(query)
# df

In [14]:
postgres_db = "db"
postgres_user = "stelligence"
postgres_password = "Stell123!"
postgres_host = "192.168.15.34"

try:
    connection_string = f"postgresql://{postgres_user}:{postgres_password}@{postgres_host}/{postgres_db}"
    engine = create_engine(connection_string)
    df = pd.read_sql(query, engine)

    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)
    pd.set_option('display.max_colwidth', None)
    
except Exception as e:
    print(f"An error occurred: {e}")


In [16]:
df.head()

,form_id,form_id_seq,employer_id,group_id,alien_id,job_01,form_status_id,alien_has_permit,alien_permit_no,alien_stay_permis_seq,end_dt_sla_doe,is_accept_work,batch
0,67112500000002-0001,1,RE1719214529010,67112500000002,RA1719214334509,งาน,WCOCAS,N,None,1.0,NaT,Y,2024-07-10
1,67113100000006-0001,1,RE1719214529010,67113100000006,None,เรื่อยๆ,AP,N,None,1.0,2024-06-26 15:22:31.980329,Y,2024-07-10
2,67113100000003-0001,1,RE1719214529010,67113100000003,None,ขุดหลุม,AP,N,None,1.0,2024-06-26 15:33:26.658998,Y,2024-07-10
3,FRM1719410342251,1,RE1719313836700,67115000000007,RA1719314043164,กรรมกร,WP,-,0,7.0,NaT,Y,2024-07-10
4,FRM1719410939563,1,RE1719313836700,67115000000019,RA1719314043164,กรรมกร,WP,-,0,19.0,NaT,Y,2024-07-10


In [17]:
df.columns

Index(['form_id', 'form_id_seq', 'employer_id', 'group_id', 'alien_id',
       'job_01', 'form_status_id', 'alien_has_permit', 'alien_permit_no',
       'alien_stay_permis_seq', 'end_dt_sla_doe', 'is_accept_work', 'batch'],
      dtype='object')

In [ ]:
query = """ 
SELECT *
FROM landing.forms
"""
df = db_manager.query_postgres(query)
df